<a href="https://colab.research.google.com/github/kmk4444/Retrieval-augmented-generation/blob/main/RAG_app_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Requirements.txt**

In [1]:
!touch requirements.txt
!echo langchain >> requirements.txt
!echo langchain-openai >> requirements.txt
!echo langchain-google-genai >> requirements.txt
!echo langchain_experimental >> requirements.txt
!echo openai >> requirements.txt
!echo anthropic >> requirements.txt
!echo cohere >> requirements.txt
!echo streamlit==1.29.0 >> requirements.txt
!echo python-dotenv >> requirements.txt
!echo beautifulsoup4 >> requirements.txt
!echo faiss-cpu >> requirements.txt
!echo pypdf >> requirements.txt
!echo unstructured >> requirements.txt
!echo networkx >> requirements.txt
!echo openpyxl >> requirements.txt
!echo rapidocr-onnxruntime >> requirements.txt


**terminal / bash komutu**


In [2]:
pip install -r requirements.txt

In [12]:
#raghelper.py
%%writefile raghelper.py
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS # it is vector database. we will use FAISS database. It is created by Facebook.
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import CohereEmbeddings
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
import os
from dotenv import load_dotenv

#load_dotenv()
#my_key_openai = os.getenv("openai_apikey")
#my_key_google = os.getenv("google_apikey")
#my_key_cohere = os.getenv("cohere_apikey")
#my_key_hf = os.getenv("huggingface_access_token")
my_key_openai="----"
my_key_google="----"
my_key_cohere="---"
my_key_hf="---"

llm_gemini = ChatGoogleGenerativeAI(google_api_key=my_key_google, model="gemini-pro")

embeddings = OpenAIEmbeddings(api_key=my_key_openai)

def ask_gemini(prompt):
  AI_Response = llm_gemini.invoke(prompt)

  return AI_Response.content

def rag_with_url(target_url,prompt):
  #load url
  loader = WebBaseLoader(target_url)
  raw_documents = loader.load() # raw_documents is document list (list[document])

  #create chunk using splitters
  #below code can run without any parameters but to reach the best answer you should try different parameters.
  #  text_splitter = RecursiveCharacterTextSplitter()
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=1000,
      chunk_overlap=0,
      length_function=len # it provides that how to estimate lenght of chunk. we will use len because of python list rule.
  )

  # we run split_documents method which is inside of text_splitter and save to splitted_document
  splitted_document = text_splitter.split_documents(raw_documents)

  # we need to save splited documents in vector database.
  # FAISS create vectors from our documents using our embedding model and save vectordatabase.
  vectorstore = FAISS.from_documents(splitted_document, embeddings)
  # we reach as_retriever method in vectorstore. We convert vector store into retriever.
  # retriever won't retrieve all vectors, it retrieves vector considering context similarity among vector and prompt.
  #mostly, similarity is estimated by cosine similarity.
  retriever = vectorstore.as_retriever()
  relevant_documents = retriever.get_relevant_documents(prompt) # it retrieves the most similar document accordingly our prompt. It can be 3,4 or more documents. Default value is 4.

  context_data =""

  #Also, we use build function instead of the below code.
  for document in relevant_documents:
    context_data = context_data + " " + document.page_content # document has two features which are metada and page_content.
    #because the main data is in context_data.
  final_prompt = f"""Şöyle bir sorum var: {prompt}
  Bu soruyu yanıtlamak için elimizde şu bilgiler var: {context_data} .
  Bu sorunun yanıtını vermek için yalnızca sana burada verdiğim eldeki bilgileri kullan. Bunların dışına asla çıkma.
  """
  AI_Response = llm_gemini.invoke(prompt)

  return AI_Response.content

def rag_with_pdf(filepath, prompt):

    loader = PyPDFLoader(filepath)

    raw_documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=0,
        length_function=len
    )


    splitted_documents = text_splitter.split_documents(raw_documents)

    vectorstore = FAISS.from_documents(splitted_documents, embeddings)
    retriever = vectorstore.as_retriever()

    relevant_documents = retriever.get_relevant_documents(prompt)

    context_data = ""

    for document in relevant_documents:
        context_data = context_data + " " + document.page_content

    final_prompt = f"""Şöyle bir sorum var: {prompt}
    Bu soruyu yanıtlamak için elimizde şu bilgiler var: {context_data} .
    Bu sorunun yanıtını vermek için yalnızca sana burada verdiğim eldeki bilgileri kullan. Bunların dışına asla çıkma.
    """

    AI_Response = llm_gemini.invoke(final_prompt)

    return AI_Response.content, relevant_documents

Overwriting raghelper.py


In [13]:
%%writefile rag.py
#Streamlit
import streamlit as st
import raghelper

st.set_page_config(page_title="Langchain ile Bellek Genişletme",layout="wide")
st.title("Langchain ile Bellek Genişletme: URL")
st.divider()

col_input, col_rag, col_normal = st.columns([1,2,2])

with col_input:
  target_url = st.text_input(label="İşlenecek Web Adresinizi Giriniz:")
  st.divider()
  prompt=st.text_input(label="Sorunuzu Giriniz:", key="url_prompt")
  st.divider()
  submit_btn = st.button(label="Sor", key="url_button")
  st.divider()

  if submit_btn:
    with col_rag:
      with st.spinner("Yanıt Hazırlanıyor...."):
        st.success("YANIT - Bellek Genişletme Devrede")
        st.markdown(raghelper.rag_with_url(target_url=target_url,prompt=prompt))
        st.divider()

    with col_normal:
      with st.spinner("Yanıt Hazırlanıyor...."):
        st.info("YANIT - Bellek Genişletme Devre Dışı") # info provides blue
        st.markdown(raghelper.ask_gemini(prompt=prompt))
        st.divider()

st.title("Langchain ile Bellek Genişletme: PDF")
st.divider()

col_input, col_rag, col_normal = st.columns([1,2,2])

with col_input:
    selected_file = st.file_uploader(label="İşlenecek Dosyayı Seçiniz", type="pdf")
    st.divider()
    prompt = st.text_input(label="Sorunuzu Giriniz:", key="pdf_prompt")
    st.divider()
    submit_btn = st.button(label="Sor", key="pdf_button")
    st.divider()


if submit_btn:
  with col_rag:
    with st.spinner("Yanıt Hazırlanıyor..."):
      st.success("YANIT - Bellek Genişletme Devrede")
      #AI_Response, relevant_documents = raghelper.rag_with_pdf(filepath=f"content/drive/MyDrive/{selected_file.name}", prompt=prompt) # we cannot upload document because of streamlit error on google colab but you can write this code in your local computer.
      AI_Response, relevant_documents = raghelper.rag_with_pdf(filepath ="/content/drive/MyDrive/gelecek.pdf", prompt=prompt)
      st.markdown(AI_Response)
      st.divider()
      for doc in relevant_documents:
        st.caption(doc.page_content)
        st.markdown(f"Kaynak: {doc.metadata}")
        st.divider()

  with col_normal:
    with st.spinner("Yanıt Hazırlanıyor..."):
      st.info("YANIT - Bellek Genişletme Devre Dışı")
      st.markdown(raghelper.ask_gemini(prompt=prompt))
      st.divider()

Overwriting rag.py


In [14]:
!npm install localtunnel
!streamlit run /content/rag.py &>/content/logs.txt &
!npx localtunnel --port 8501

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.537s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details
npx: installed 22 in 4.104s
your url is: https://crazy-mice-attack.loca.lt
^C
